### Creating a function that pulls data from a company’s income statement

In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("FINANCIAL_API_KEY")
#Function that returns income statement data
def get_data2(stock):
    data = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{stock}?period=quarter&limit=400&apikey={api_key}")

    data = data.json()
# Pull data from the previous four quarters. The 0 refers to the most previous quarter etc. 
    data = data[0:5]
    
    return data

In [2]:
get_data2("PRU")

[{'date': '2020-06-30',
  'symbol': 'PRU',
  'fillingDate': '2020-08-06',
  'acceptedDate': '2020-08-06 16:15:01',
  'period': 'Q2',
  'revenue': 11681000000,
  'costOfRevenue': 0.0,
  'grossProfit': 11681000000,
  'grossProfitRatio': 1.0,
  'researchAndDevelopmentExpenses': 0.0,
  'generalAndAdministrativeExpenses': 3878000000,
  'sellingAndMarketingExpenses': 0.0,
  'otherExpenses': 0.0,
  'operatingExpenses': 14447000000,
  'costAndExpenses': 14447000000,
  'interestExpense': 0.0,
  'depreciationAndAmortization': 54000000,
  'ebitda': -2240000000,
  'ebitdaratio': -0.191764,
  'operatingIncome': -2332000000,
  'operatingIncomeRatio': -0.19964,
  'totalOtherIncomeExpensesNet': 1474000000,
  'incomeBeforeTax': -2332000000,
  'incomeBeforeTaxRatio': -0.19964,
  'incomeTaxExpense': 115000000,
  'netIncome': -2409000000,
  'netIncomeRatio': -0.206232,
  'eps': -6.10492,
  'epsdiluted': -6.10492,
  'weightedAverageShsOut': 394600000,
  'weightedAverageShsOutDil': 394600000,
  'link': 'htt

###  Calculates financial growth ,revenue, net income, EPS, and EBITDA growth

In [3]:
def growth(stock):
    #call the get_data2 function and store the results in a       
    #variable, data 2.
    data = get_data2(stock)
    
    #revenue growth
    rev_new = data[0]['revenue']
    rev_old = data[-1]['revenue']
    rev_change = 100*((rev_new-rev_old)/rev_old)
    #net income growth
    income_new = data[0]['netIncome']
    income_old = data[-1]['netIncome']
    income_change = 100*((income_new-income_old)/income_old)
    
    #EPS growth
    eps_new = data[0]['eps']
    eps_old = data[-1]['eps']
    eps_change = 100*((eps_new-eps_old)/eps_old)
    
    #EBITDA
    ebitda_new = data[0]['ebitda']
    ebitda_old = data[-1]['ebitda']
    ebitda_change = 100*((ebitda_new-ebitda_old)/ebitda_old)
    
    return (rev_change, income_change, eps_change, ebitda_change)

### Retrieving opreating performance metrics from different securities 

In [4]:
stocks = ['PTON', 'MSFT', 'AAPL', 'AMD']
#store financials in a variable, data.
data = map(growth, stocks)
# populate DataFrame with data variable and create columns:
df1 = pd.DataFrame(data, columns=['Rev_Growth', 'Income_Growth', 'EPS_Growth', 'ebitda_Growth'], index=stocks)
data

df1.reset_index(inplace=True)
df1.rename(columns={'index': 'Symbol'}, inplace=True)
#display output
df1

Symbol  Rev_Growth  Income_Growth  EPS_Growth  ebitda_Growth
0   PTON   62.341840    -224.548736 -221.291757    -298.492462
1   MSFT   12.400545      30.108635   32.856782      13.667608
2   AAPL    1.027483      -7.401724   -3.555244     -13.118910
3    AMD   55.524708    -100.000000 -100.000000     -55.450237

In [5]:
def dcf(stock):
    data = requests.get(f"https://financialmodelingprep.com/api/v3/discounted-cash-flow/{stock}?apikey={api_key}")
    data = data.json()
    dcf = data[0]['dcf']
    current_price = data[0]['Stock Price']
    return (dcf, current_price)

In [6]:
stocks = ['PTON', 'MSFT', 'AAPL', 'AMD']
data = map(dcf, stocks)
df2 = pd.DataFrame(data, columns=['dcf', 'current price'], index=stocks)
df2.reset_index(inplace=True)
df2.rename(columns={'index': 'symbol'}, inplace=True)
#Create a column that indicates under or over-valued
df2['value'] = df2.apply(lambda x: 'Under-valued' if x['dcf']>x['current price'] else 'Over-valued',axis=1)
#display results
df2

symbol         dcf  current price         value
0   PTON  105.476533         100.79  Under-valued
1   MSFT  218.277018         216.51  Under-valued
2   AAPL  121.428846         119.26  Under-valued
3    AMD   82.738929          81.43  Under-valued

In [7]:
# Join industry peers
df = (
    pd.concat([df1,df2 ], axis='columns', join='inner').dropna()
)
df.head()

Symbol  Rev_Growth  Income_Growth  EPS_Growth  ebitda_Growth symbol  \
0   PTON   62.341840    -224.548736 -221.291757    -298.492462   PTON   
1   MSFT   12.400545      30.108635   32.856782      13.667608   MSFT   
2   AAPL    1.027483      -7.401724   -3.555244     -13.118910   AAPL   
3    AMD   55.524708    -100.000000 -100.000000     -55.450237    AMD   

          dcf  current price         value  
0  105.476533         100.79  Under-valued  
1  218.277018         216.51  Under-valued  
2  121.428846         119.26  Under-valued  
3   82.738929          81.43  Under-valued